<a href="https://colab.research.google.com/github/bozkurtmert0/text-generator/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [6]:
path = "/content/Arthur-Schopenhauer-Quotes.txt"
text = open(path, 'rb').read().decode(encoding='unicode_escape')

In [7]:
print(text[:250])

It is difficult to find happiness within oneself, but it is impossible to find it anywhere else.
All truth passes through three stages. First, it is ridiculed. Second, it is violently opposed. Third, it is accepted as being self-evident.
The wise h


In [8]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

75 unique characters


In [9]:
words = sorted(set(text))

char2idx = {u:i for i, u in enumerate(words)}
idx2char = np.array(words)

def text2num(yazi):
  return np.array([char2idx[c] for c in yazi])
  
num_text = text2num(text)


In [10]:
print(text[:15])

It is difficult


In [11]:
print(text2num(text[:15]))

[27 63  2 52 62  2 47 52 49 49 52 46 64 55 63]


In [12]:
def num2text(nums):
  try:
    nums = nums.numpy()
  except:
    pass
  return ''.join(idx2char[nums])


In [13]:
print(num2text(text2num(text[:15])))

It is difficult


In [14]:
seq_length = 100  
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(num_text)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [15]:
def split_input_target(chunk):
    input_text = chunk[:-1]  
    target_text = chunk[1:] 
    return input_text, target_text
dataset = sequences.map(split_input_target)

In [16]:
BATCH_SIZE = 64
VOCAB_SIZE = len(words) 
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           19200     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 75)            76875     
                                                                 
Total params: 5,343,051
Trainable params: 5,343,051
Non-trainable params: 0
_________________________________________________________________


In [19]:
pred = example_batch_predictions[0]
time_pred = pred[0]

Random Prediction

In [20]:
sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = num2text(sampled_indices)
predicted_chars

'VPC]K\x97\rFBRDO«oP\x858;Dlo(J!I,ivUAi][(\x928AFA?y!\rxOqIiI\r\xadKBLjs\xad1\r-hB1g\nGalvYfjCOBcWac«j?(0f"jNvrSMLD8NsVMD'

In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [22]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [23]:
history = model.fit(data, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
10/10 [==============================] - 3s 84ms/step - loss: 3.4928
Epoch 2/100
10/10 [==============================] - 1s 75ms/step - loss: 2.9891
Epoch 3/100
10/10 [==============================] - 1s 91ms/step - loss: 2.9240
Epoch 4/100
10/10 [==============================] - 1s 82ms/step - loss: 2.8275
Epoch 5/100
10/10 [==============================] - 1s 80ms/step - loss: 2.6643
Epoch 6/100
10/10 [==============================] - 1s 76ms/step - loss: 2.5068
Epoch 7/100
10/10 [==============================] - 1s 79ms/step - loss: 2.4027
Epoch 8/100
10/10 [==============================] - 1s 73ms/step - loss: 2.3299
Epoch 9/100
10/10 [==============================] - 1s 76ms/step - loss: 2.2679
Epoch 10/100
10/10 [==============================] - 1s 76ms/step - loss: 2.2166
Epoch 11/100
10/10 [==============================] - 1s 81ms/step - loss: 2.1702
Epoch 12/100
10/10 [==============================] - 1s 74ms/step - loss: 2.1273
Epoch 13/100
10/10 [=====

In [24]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [25]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [28]:
def generate_text(model, start_string):
  
  num_generate = 100

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
    
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

inp = input("Input for generator: ")
print(generate_text(model, inp))

Input for generator: Life is
Life is no harlot who throws her arms round the neck of him who does not desire her; on the contrary, shoul
